In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [17]:
from sklearn.feature_selection import chi2

import argparse
import pandas as pd
import sys
import numpy as n
import pdb
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.utils import shuffle
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import codecs
import operator
import sklearn
from collections import defaultdict
from nltk.tokenize import TweetTokenizer


In [19]:
from keras.models import Sequential
from keras import layers


In [20]:

from keras.callbacks import EarlyStopping


In [21]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from keras.layers import Bidirectional

In [23]:
df=pd.read_csv("celebrity_datas.csv",usecols=['text','gender'])
df.text=df.text.astype(str)
df.gender=df.gender.astype(str)

In [ ]:
X = df.text
Y =  df.gender
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [ ]:
df=pd.read_csv("celebrity_datas.csv",usecols=['text','occupation'])
df.text=df.text.astype(str)
df.occupation=df.occupation.astype(str)

NameError: ignored

In [ ]:
df.loc[df['occupation'] == 'science', 'occupation'] = 'politics'
df.loc[df['occupation'] == 'manager', 'occupation'] = 'politics'
df.loc[df['occupation'] == 'professional', 'occupation'] = 'politics'
df.loc[df['occupation'] == 'religious', 'occupation'] = 'politics'

In [ ]:
df.occupation.value_counts()

sports       13481
performer     9899
creator       5475
politics      4981
Name: occupation, dtype: int64

In [ ]:
X = df.text
Y =  df.occupation
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)


MAX_NB_WORDS = 1000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(df['occupation']).values
print('Shape of label tensor:', Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 7
batch_size = 128

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.15,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Found 388971 unique tokens.
Shape of data tensor: (33836, 250)
Shape of label tensor: (33836, 4)
Epoch 1/7
191/191 [==============================] - 138s 707ms/step - loss: 1.2461 - accuracy: 0.4328 - val_loss: 0.9317 - val_accuracy: 0.6342
Epoch 2/7
191/191 [==============================] - 135s 709ms/step - loss: 0.9459 - accuracy: 0.6295 - val_loss: 0.9036 - val_accuracy: 0.6472
Epoch 3/7
191/191 [==============================] - 135s 708ms/step - loss: 0.9166 - accuracy: 0.6487 - val_loss: 0.9091 - val_accuracy: 0.6437
Epoch 4/7
191/191 [==============================] - 135s 707ms/step - loss: 0.8751 - accuracy: 0.6619 - val_loss: 0.8747 - val_accuracy: 0.6676
Epoch 5/7
191/191 [==============================] - 135s 706ms/step - loss: 0.8490 - accuracy: 0.6784 - val_loss: 0.8824 - val_accuracy: 0.6637
Epoch 6/7
191/191 [==============================] - 135s 706ms/step - loss: 0.8400 - accuracy: 0.6803 - val_loss: 0.8758 - val_accuracy: 0.6634
Epoch 7/7
159/159 [==============

In [ ]:
df=pd.read_csv("celebrity_datas.csv",usecols=['text','fame'])
df.text=df.text.astype(str)
df.fame=df.fame.astype(str)

ParserError: ignored

In [ ]:
X = df.text
Y =  df.fame
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

df.fame.value_counts()

MAX_NB_WORDS = 1000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(df['fame']).values
print('Shape of label tensor:', Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 128

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.15,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Found 388971 unique tokens.
Shape of data tensor: (33836, 250)
Shape of label tensor: (33836, 3)
Epoch 1/10
191/191 [==============================] - 150s 766ms/step - loss: 0.7772 - accuracy: 0.7292 - val_loss: 0.6736 - val_accuracy: 0.7487
Epoch 2/10
191/191 [==============================] - 145s 759ms/step - loss: 0.6695 - accuracy: 0.7454 - val_loss: 0.6611 - val_accuracy: 0.7459
Epoch 3/10
191/191 [==============================] - 145s 759ms/step - loss: 0.6499 - accuracy: 0.7448 - val_loss: 0.6747 - val_accuracy: 0.7427
Epoch 4/10
191/191 [==============================] - 145s 758ms/step - loss: 0.6273 - accuracy: 0.7508 - val_loss: 0.6671 - val_accuracy: 0.7404
Epoch 5/10
159/159 [==============================] - 7s 42ms/step - loss: 0.6726 - accuracy: 0.7447
Test set
  Loss: 0.673
  Accuracy: 0.745


In [ ]:
model.summary()

In [24]:
df=pd.read_csv("celebrity_datas.csv",usecols=['text','birthyear'])
df.text=df.text.astype(str)
df.birthyear=df.birthyear.astype(str)


In [27]:
X = df.text
Y =  df.birthyear
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)
max_words = 100
max_len = 150
tok = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,100,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('softmax')(layer)
    
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 100)          10000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                42240     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0   